In [48]:
import numpy as np
import pandas as pd
import sys
import os
import duckdb
sys.path.append('..')
from helper import *
setwd()

Objects = Objects()

In [2]:
con = duckdb.connect(database = "./data/exploitation/exploitation.db", read_only=False)

for datasource in Objects:
    id = datasource['id']
    con_trust = duckdb.connect(database = f"data/trusted/db_{id}.db")
    con_trust.execute("EXPORT DATABASE './temp'")
    con_trust.close()
    con.execute(f"IMPORT DATABASE './temp'")

con.execute("SHOW TABLES").fetchall()

[('hospitals',), ('housing',), ('schools',)]

In [3]:
!rm ./temp/*

In [50]:
# lat long
con.execute("CREATE OR REPLACE TABLE pos_house AS SELECT lat, long FROM housing;")
con.execute("SELECT * FROM pos_house;").fetchdf().head

<bound method NDFrame.head of             lat     long
0       29.5920 -98.4700
1       29.5042 -98.5697
2       29.3595 -98.4499
3       29.4682 -98.5298
4       29.3466 -98.5074
...         ...      ...
384972  36.1151 -94.0489
384973  36.1151 -94.0489
384974  36.3169 -94.1545
384975  36.2200  94.1248
384976  36.1043 -94.1550

[384977 rows x 2 columns]>

In [5]:
con.execute("""SELECT lat, long, COUNT(*) FROM pos_house GROUP BY lat, long""").fetchdf().head()

,lat,long,count_star()
0,44.0223,-123.081,58
1,44.5255,-122.910,33
2,44.0823,-123.111,71
3,44.0836,-123.161,1
4,44.0606,-123.000,4


In [52]:
%%time
con.execute(f"""
create or replace table pos_house_hospital as (
with H as (SELECT * FROM pos_house),
S as (SELECT X, Y, objectid, type, beds, owner from hospitals)
SELECT
    lat, long,
    count(S.objectid) as num_hospitals,
    sum(S.beds) as num_beds,
    sum(if(S.type='GENERAL ACUTE CARE', 1, 0)) as hospital_type_general,
    sum(if(S.type='CRITICAL ACCESS', 1, 0)) as hospital_type_critical,
    sum(if(S.type='PSYCHIATRIC', 1, 0)) as hospital_type_psychiatric,
    sum(if(S.type='LONG TERM CARE', 1, 0)) as hospital_type_longterm,
    sum(if(S.type='CHILDREN', 1, 0)) as hospital_type_children,
    sum(if(S.owner in('GOVERNMENT - DISTRICT/AUTHORITY', 'GOVERNMENT - FEDERAL',
        'GOVERNMENT - LOCAL', 'GOVERNMENT - STATE'), 1, 0)) as government_hospital,
    sum(if(S.owner = 'NON-PROFIT', 1, 0)) as nonprofit_hospital,
    sum(if(S.owner='PROPRIETARY', 1, 0)) as private_hospital,
    
FROM H left join S on (
    (H.long - S.X)*(H.long - S.X) + (H.lat - S.Y)*(H.lat - S.Y) < 0.01
)
GROUP BY lat, long
)
""").fetchall()

CPU times: total: 3min 30s
Wall time: 1min 12s


[(79690,)]

In [53]:
con.execute("SELECT * FROM pos_house_hospital;").fetchdf()

,lat,long,num_hospitals,num_beds,hospital_type_general,hospital_type_critical,hospital_type_psychiatric,hospital_type_longterm,hospital_type_children,government_hospital,nonprofit_hospital,private_hospital
0,45.6721,-122.5470,3,1080.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
1,45.5408,-122.5780,6,458.0,5.0,0.0,0.0,1.0,0.0,1.0,4.0,1.0
2,45.6359,-122.5160,22,7920.0,22.0,0.0,0.0,0.0,0.0,0.0,22.0,0.0
3,45.5415,-122.5760,12,916.0,10.0,0.0,0.0,2.0,0.0,2.0,8.0,2.0
4,45.5556,-122.5560,4,1050.0,3.0,0.0,0.0,1.0,0.0,0.0,3.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
79685,36.0480,-95.7835,6,442.0,3.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0
79686,46.7327,-120.7140,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
79687,37.4580,-77.4700,4,473.0,3.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0
79688,36.2809,-95.8639,2,109.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [54]:
df_schools = con.execute("SELECT * FROM schools;").fetchdf()
df_schools["LEVEL_"].replace("N","unknown", inplace=True)
df_schools["LEVEL_"].replace("1","preschool", inplace=True)
df_schools["LEVEL_"].replace("2","elementary_school", inplace=True)
df_schools["LEVEL_"].replace("3","middle_school", inplace=True)
df_schools["LEVEL_"].replace("4","high_school", inplace=True)
con.execute("CREATE OR REPLACE TABLE schools AS SELECT * FROM df_schools;")

In [55]:
con.execute("SELECT * FROM schools;").fetchdf().head()

,X,Y,OBJECTID,NCESID,NAME,ADDRESS,CITY,STATE,ZIP,ZIP4,...,VAL_METHOD,VAL_DATE,WEBSITE,LEVEL_,ENROLLMENT,ST_GRADE,END_GRADE,DISTRICTID,FT_TEACHER,SHELTER_ID
0,-97.855472,36.405877,91704,401092000509,COOLIDGE ES,1515 EAST ASH AVENUE,ENID,OK,73701,3701,...,IMAGERY/OTHER,2016-08-31,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,preschool,517,PK,05,4010920,29,NOT AVAILABLE
1,-75.263245,40.221458,91705,421728004831,GWYN-NOR EL SCH,139 HANCOCK ROAD,NORTH WALES,PA,19454,NOT AVAILABLE,...,IMAGERY/OTHER,2016-09-12,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,preschool,581,KG,06,4217280,35,NOT AVAILABLE
2,-89.911159,35.060253,91706,470014801097,KNIGHT ROAD ELEMENTARY,3237 KNIGHT RD,MEMPHIS,TN,38118,NOT AVAILABLE,...,IMAGERY,2016-01-07,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,preschool,516,PK,05,4700148,29,NOT AVAILABLE
3,-86.220500,35.297310,91707,470129000069,NORTH LAKE ELEMENTARY,10626 OLD TULLAHOMA RD,TULLAHOMA,TN,37388,NOT AVAILABLE,...,IMAGERY/OTHER,2016-09-19,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,preschool,288,PK,05,4701290,19,10795577
4,-77.424665,37.565102,91708,510324001389,OVERBY-SHEPPARD ELEM,2300 1ST AVE,RICHMOND,VA,23222,4699,...,IMAGERY/OTHER,2016-10-27,http://nces.ed.gov/GLOBALLOCATOR/sch_info_popu...,preschool,454,PK,05,5103240,24,NOT AVAILABLE


In [56]:
%%time
con.execute(f"""
create or replace table pos_house_schools as (
with H as (SELECT * FROM pos_house),
S as (SELECT X, Y, OBJECTID, POPULATION, LEVEL_ from schools)
SELECT
    lat, long,
    count(S.OBJECTID) as num_schools,
    sum(S.POPULATION) as num_students,
    sum(if(S.LEVEL_='preschool', 1, 0)) as preschool,
    sum(if(S.LEVEL_='elementary_school', 1, 0)) as elementary_school,
    sum(if(S.LEVEL_= 'middle_school', 1, 0)) as middle_school,
    sum(if(S.LEVEL_= 'high_school', 1, 0)) as high_school
    
FROM H left join S on (
    (H.long - S.X)*(H.long - S.X) + (H.lat - S.Y)*(H.lat - S.Y) < 0.01
    )
GROUP BY lat, long
)
""").fetchall()

CPU times: total: 48min 19s
Wall time: 16min 52s


[(79690,)]

In [57]:
con.execute("SELECT * FROM pos_house_schools;").fetchdf()

,lat,long,num_schools,num_students,preschool,elementary_school,middle_school,high_school
0,45.6721,-122.5470,207,130908.0,99.0,33.0,51.0,24.0
1,45.5313,-122.6450,96,49609.0,67.0,10.0,14.0,4.0
2,45.6537,-122.6670,134,68676.0,74.0,16.0,22.0,20.0
3,45.5396,-122.6600,92,48294.0,62.0,10.0,14.0,5.0
4,45.6914,-122.5800,1584,949014.0,748.0,198.0,396.0,242.0
...,...,...,...,...,...,...,...,...
79685,36.0480,-95.7835,45,31682.0,30.0,6.0,2.0,3.0
79686,46.7327,-120.7140,7,2402.0,3.0,2.0,2.0,0.0
79687,37.4580,-77.4700,57,30755.0,31.0,10.0,6.0,5.0
79688,36.2809,-95.8639,17,13002.0,11.0,3.0,2.0,1.0


In [61]:
%%time 
con.execute(f"""
create or replace table houses as (
with H as (SELECT * FROM housing),
T1 as (SELECT * from pos_house_hospital),
T2 as (SELECT * from pos_house_schools)
SELECT *
FROM H, T1, T2
WHERE H.long == T1.long and H.lat == T1.lat
      and T1.long == T2.long and T1.lat == T2.lat
)
""").fetchall()
#pos_house_schools
#T2 as (SELECT * from pos_house_schools)
#FROM H left join T1 on ( H.long == T1.long and H.lat == T1.lat)

CPU times: total: 6.12 s
Wall time: 16.8 s


[(382919,)]

In [62]:
df = con.execute("SELECT * FROM houses;").fetchdf()

In [60]:
columns_names = list(df.columns)
print(columns_names)

['id', 'url', 'region', 'region_url', 'price', 'type', 'sqfeet', 'beds', 'baths', 'cats_allowed', 'dogs_allowed', 'smoking_allowed', 'wheelchair_access', 'electric_vehicle_charge', 'comes_furnished', 'laundry_options', 'parking_options', 'image_url', 'description', 'lat', 'long', 'state', 'lat:1', 'long:1', 'num_hospitals', 'num_beds', 'hospital_type_general', 'hospital_type_critical', 'hospital_type_psychiatric', 'hospital_type_longterm', 'hospital_type_children', 'government_hospital', 'nonprofit_hospital', 'private_hospital', 'lat:2', 'long:2', 'num_schools', 'num_students', 'preschool', 'elementary_school', 'middle_school', 'high_school']


In [66]:
con.close()

In [64]:
con.execute("select count(*) from housing").fetchall()

[(384977,)]

In [65]:
con.execute("select count(*) from houses").fetchall()

[(382919,)]